```{contents}
```
## Logging

Logging in LangGraph is the foundation of **observability, debugging, safety, auditing, and production reliability**.
It provides a structured record of **what happened, when, where, why, and with what state** during graph execution.

---

### **1. Purpose of Logging in LangGraph**

LangGraph logging answers:

| Question              | Why It Matters      |
| --------------------- | ------------------- |
| What executed?        | Debug correctness   |
| Why did it execute?   | Understand routing  |
| What state changed?   | Trace reasoning     |
| Where did it fail?    | Root-cause analysis |
| How long did it take? | Performance tuning  |
| Who approved?         | Compliance & audit  |

Without logging, complex agent graphs become **opaque and unsafe**.

---

### **2. What Is Logged**

LangGraph logs events across multiple layers.

| Layer   | Examples                    |
| ------- | --------------------------- |
| Graph   | Entry, exit, transitions    |
| Node    | Invocation, outputs, errors |
| State   | Before/after snapshots      |
| LLM     | Prompt, response, tokens    |
| Tools   | Inputs, outputs, failures   |
| Control | Routing decisions           |
| System  | Retries, timeouts           |
| Human   | Approvals, overrides        |

---

### **3. Logging Architecture**

```
Graph Execution
    ↓
Event Emitter
    ↓
Structured Logger
    ↓
Log Store (File / DB / APM)
```

Each event includes:

```json
{
  "run_id": "...",
  "node": "reason",
  "event": "node_complete",
  "timestamp": "...",
  "state_diff": {...},
  "latency_ms": 812
}
```

---

### **4. Basic Logging Setup**

```python
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("langgraph")
```

Attach to graph execution:

```python
graph.invoke(input, config={"callbacks": [logger]})
```

---

### **5. Custom Node Logging**

```python
def analyze(state):
    logger.info("Analyzing input", extra={"state": state})
    result = llm.invoke(state["text"])
    logger.info("LLM response", extra={"output": result})
    return {"analysis": result}
```

---

### **6. Structured Logging with Context**

```python
logger.info(
    "Transition",
    extra={
        "from_node": "observe",
        "to_node": "reason",
        "state_version": state["version"]
    }
)
```

Structured logs enable querying and dashboards.

---

### **7. Logging with Tracing (LangSmith)**

```python
from langchain.callbacks import LangChainTracer

tracer = LangChainTracer()
graph.invoke(input, config={"callbacks": [tracer]})
```

Provides:

* Node-level timelines
* Token usage
* Tool traces
* Error stacks
* Full state history

---

### **8. Production-Grade Logging Pipeline**

```
LangGraph → Logger → OpenTelemetry → Collector → ELK / Datadog / Grafana
```

| Component     | Role                |
| ------------- | ------------------- |
| OpenTelemetry | Distributed tracing |
| Collector     | Aggregation         |
| ELK           | Searchable logs     |
| Grafana       | Visualization       |
| Alerting      | Incident response   |

---

### **9. Log Levels & Usage**

| Level    | Usage              |
| -------- | ------------------ |
| DEBUG    | Development        |
| INFO     | Normal operations  |
| WARNING  | Recoverable issues |
| ERROR    | Failures           |
| CRITICAL | System failure     |

---

### **10. Safety & Compliance Logging**

Mandatory for production:

* State history
* Human approvals
* Model decisions
* Tool execution
* Data access

This enables:

* Auditability
* Explainability
* Regulatory compliance

---

### **11. Common Logging Patterns**

| Pattern          | Purpose               |
| ---------------- | --------------------- |
| Execution Trace  | End-to-end visibility |
| State Diff Logs  | Track reasoning       |
| Decision Logs    | Explain routing       |
| Error Logs       | Root cause            |
| Performance Logs | Optimization          |

---

### **12. Minimal Example with Logging**

```python
def step(state):
    logger.info("Before", extra={"state": state})
    state["count"] += 1
    logger.info("After", extra={"state": state})
    return state
```

---

### **13. Why Logging Is Critical for LangGraph**

LangGraph systems are:

* Stateful
* Cyclic
* Autonomous
* Multi-agent

Without logging, such systems become **unmaintainable and unsafe**.

---

### **Mental Model**

> Logging is the **black box recorder** of an intelligent system.


### Demonstration

In [1]:
# ============================
# LangGraph Logging Demo
# ============================

import logging
from typing import TypedDict
from langgraph.graph import StateGraph, END

# -------- Logging Setup --------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
)
logger = logging.getLogger("LangGraphDemo")

# -------- State Definition --------
class State(TypedDict):
    count: int

# -------- Nodes with Logging --------
def increment(state: State):
    logger.info(f"Node: increment | Before: {state}")
    new_state = {"count": state["count"] + 1}
    logger.info(f"Node: increment | After: {new_state}")
    return new_state

def check(state: State):
    logger.info(f"Node: check | State: {state}")
    done = state["count"] >= 3
    logger.info(f"Decision: done={done}")
    return {"done": done}

# -------- Graph Construction --------
builder = StateGraph(State)

builder.add_node("increment", increment)
builder.add_node("check", check)

builder.set_entry_point("increment")
builder.add_edge("increment", "check")

builder.add_conditional_edges(
    "check",
    lambda s: END if s["done"] else "increment",
    {"increment": "increment", END: END}
)

graph = builder.compile()

# -------- Execute Graph --------
result = graph.invoke({"count": 0})

print("\nFinal Result:", result)


2025-12-30 11:59:49,411 | INFO | Node: increment | Before: {'count': 0}
2025-12-30 11:59:49,413 | INFO | Node: increment | After: {'count': 1}
2025-12-30 11:59:49,415 | INFO | Node: check | State: {'count': 1}
2025-12-30 11:59:49,415 | INFO | Decision: done=False
2025-12-30 11:59:49,417 | INFO | Node: increment | Before: {'count': 1}
2025-12-30 11:59:49,417 | INFO | Node: increment | After: {'count': 2}
2025-12-30 11:59:49,418 | INFO | Node: check | State: {'count': 2}
2025-12-30 11:59:49,419 | INFO | Decision: done=False
2025-12-30 11:59:49,420 | INFO | Node: increment | Before: {'count': 2}
2025-12-30 11:59:49,422 | INFO | Node: increment | After: {'count': 3}
2025-12-30 11:59:49,424 | INFO | Node: check | State: {'count': 3}
2025-12-30 11:59:49,425 | INFO | Decision: done=True



Final Result: {'count': 3}
